In [1]:
import sqlite3
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import pickle

In [2]:
con = sqlite3.connect(r"C:\Users\reekithak\Downloads\drugs.db")

In [3]:
df = pd.read_sql("SELECT * from train_set6",con)

In [4]:
df

,cid,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,...,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,target
0,9838682,1.148385,-0.125281,-0.315079,0.304786,-0.004684,0.472062,0.177793,-0.396318,0.485568,...,-0.348502,0.256189,0.505184,-0.329519,-0.199583,0.020498,0.671782,0.045342,-0.069823,5CLS
1,10286210,1.099651,-0.145843,-0.318188,0.341535,0.198539,0.371798,0.137645,-0.386924,0.513982,...,-0.114895,0.169443,0.313858,-0.280098,-0.075921,0.067442,0.250539,-0.022959,-0.145652,5CLS
2,9900824,0.875428,-0.164715,-0.279007,0.378172,0.025336,0.351377,0.058648,-0.295366,0.486543,...,0.019188,0.131522,0.263330,-0.524534,-0.421218,-0.162838,0.395380,0.166860,-0.189730,5CLS
3,16757237,1.005219,-0.138417,-0.351153,0.346939,0.127122,0.367690,0.051513,-0.450858,0.493794,...,0.010793,-0.026575,0.298154,-0.367923,-0.103681,-0.129132,0.164891,0.113320,-0.189716,5CLS
4,16757238,0.313630,0.239549,-0.136016,0.277300,-0.076377,1.000764,0.317081,0.411524,0.601499,...,-0.742446,0.555215,0.115705,-0.275580,-0.232951,0.265975,0.332163,-0.144340,0.198719,5CLS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,44720028,1.439281,-0.000330,-0.480154,0.423505,-0.049969,0.773157,0.053941,-0.556313,0.643178,...,-0.186557,-0.130433,0.398519,-0.425930,-0.212712,-0.094322,0.443979,0.145596,-0.170606,5CLS
1106,421108,0.301669,-0.303042,0.216502,0.041453,-0.785708,-0.815729,-0.306233,-0.721186,-0.366644,...,1.063504,-1.322625,0.125840,-0.247714,-1.238628,-0.520483,0.086367,1.655578,-0.480996,"5CLS,2B3H"
1107,683337,0.096854,-0.213209,0.359088,0.161102,-0.874019,-0.398242,-0.154979,-0.760396,-0.279279,...,1.108250,-1.170013,0.070543,0.120076,-0.802295,-0.234409,-0.225013,1.884845,-0.449649,"5CLS,2B3H"
1108,699784,0.164133,-0.181320,0.256538,0.162686,-0.781069,-0.567260,-0.203030,-0.648409,-0.257399,...,1.071407,-1.236650,-0.051752,-0.040265,-1.020779,-0.391913,-0.042247,1.817659,-0.510953,"5CLS,2B3H"


In [5]:
cids = df["cid"].unique()
unique_pbdid = set(",".join(df["target"].values.flatten().tolist()).split(","))
pbdid_dict = {v:k for k, v in dict(enumerate(unique_pbdid)).items()}

In [6]:
X = []
y = []
print("Preparing Data")
for cid in tqdm(cids, total=len(cids), leave=False):
    indices = np.where(df["cid"] == cid)[0]
    xi = df.iloc[indices, 1:-1].values
    X.append(xi)
    yi = np.zeros(len(pbdid_dict))
    targets = ",".join(df.iloc[indices, -1].values.tolist()).split(",")
    targets = [pbdid_dict[_id] for _id in targets if _id != ""]
    yi[targets] = 1
    y.append(yi)

X = np.array(X).reshape(-1, 32)
scaler = StandardScaler()
scaler.fit(X)
y = np.array(y)

np.save('X.npy',X)
np.save('y.npy',y)

 10%|██▊                          | 107/1110 [00:00<00:00, 1068.62it/s]

Preparing Data


In [17]:
X.shape

(509983, 32)

In [2]:
X = np.load('X.npy') ; y = np.load("y.npy")

In [7]:
y.shape

(1110, 149)

In [9]:
pd.DataFrame(y)

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X.shape, y.shape

((1110, 32), (1110, 149))

In [10]:

x_t, x_v, y_t, y_v = train_test_split(X, y, test_size=0.1, random_state=42)

In [13]:
x_t.shape , y_t.shape

((999, 32), (999, 149))

In [14]:
svc = OneVsRestClassifier(SVC())

In [15]:
svc.fit(x_t,y_t)

OneVsRestClassifier(estimator=SVC())

In [16]:
svc

OneVsRestClassifier(estimator=SVC())

In [16]:
fin_list = []

In [ ]:
import winsound 
  
freq = 100
dur = 50
  
# loop iterates 5 times i.e, 5 beeps will be produced. 
for i in range(0, 5):     
    winsound.Beep(freq, dur)     
    freq+= 100
    dur+= 50

In [7]:
y.shape

(1140831971,)

In [28]:
X.shape , y.shape

((1140831971,), (509983, 32))

In [9]:
def batches(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]


In [37]:
from sklearn.linear_model import SGDClassifier
import random
import pickle
import time
import tqdm
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [14]:
clf2 = SGDClassifier(loss='log')

In [15]:
shuffledRange = list(range(len(X)))

In [25]:
len(shuffledRange)

509983

In [27]:
n_iter = 1

In [29]:
t1 = time.time()
for n in range(n_iter):
    random.shuffle(shuffledRange)
    shuffledX = [X[i] for i in shuffledRange]
    shuffledY = [y[i] for i in shuffledRange]
    print("ITER ",n)
    count = 0
    print("Init_Batches = ",len(shuffledX))
    for batch in batches(range(len(shuffledX)), 10000):
        clf2.partial_fit(shuffledX[batch[0]:batch[-1]+1], shuffledY[batch[0]:batch[-1]+1], classes=np.unique(y))
        print('Batch ',count)
        count+=1
pickle.dump(clf2,open("final_model.sav",'wb'))
print(time.time()-t1)

ITER  0
Init_Batches =  509983
Batch  0
Batch  1
Batch  2
Batch  3
Batch  4
Batch  5
Batch  6
Batch  7
Batch  8
Batch  9
Batch  10
Batch  11
Batch  12
Batch  13
Batch  14
Batch  15
Batch  16
Batch  17
Batch  18
Batch  19
Batch  20
Batch  21
Batch  22
Batch  23
Batch  24
Batch  25
Batch  26
Batch  27
Batch  28
Batch  29
Batch  30
Batch  31
Batch  32
Batch  33
Batch  34
Batch  35
Batch  36
Batch  37
Batch  38
Batch  39
Batch  40
Batch  41
Batch  42
Batch  43
Batch  44
Batch  45
Batch  46
Batch  47
Batch  48
Batch  49
Batch  50
2884.693594455719


In [31]:
#Simple Evaluation


In [34]:
clf2.score(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [1140831971, 509983]